In [5]:
library("rjson")
library("seqinr")
library("seqRFLP")
options(warn=-1)
library("protr")
library('DescTools')
library(jsonlite)
library(data.table)
library("writexl")


#### import metapredict indexes


In [5]:
json_data_metapredict<- fromJSON(txt='./metapredict/metapredict_least20_concIDRs.json')

#### import Quazi IDR Finder Table

In [6]:
outcome_IDP_predictor_counts<- fromJSON("./Quasi_IDR_Finder/output_quaziIDRFinder_canonical_all_qIDR.json")


In [7]:
outcome_IDP_predictor=outcome_IDP_predictor_counts[lengths(outcome_IDP_predictor_counts) != 0]

#### get protein names that have longer than 100 aas

In [6]:
dict_keys_lenlongerthan100<- fromJSON('./HG38_pep/longerthan100dic.json')

#### intersect Quazi IDR found keys and metapredict prediction found keys

In [9]:
int_keys=intersect(names(outcome_IDP_predictor),names(json_data_metapredict))

#### check if metapredict and Quasi IDR regions overlap

In [12]:

count=0



overlap_outcome=list()
overlapped_region=list()
overlapped_percentage=list()


for(i in 1:length(int_keys)){


     a=int_keys[i]

     idp_values=outcome_IDP_predictor[[a]]

     metapredict_values=json_data_metapredict[[a]]

     cut=idp_values[1:2]


     if (is.null(metapredict_values)) { 


         overlap_outcome[[i]] <- 'no'

         overlapped_region[[i]]=NULL

         overlapped_percentage[[i]]=0

         next



        }



    for(y in seq(from=1, to=length(metapredict_values), by=2)){

        sequence=c(metapredict_values[y]+1,metapredict_values[y+1])

        perc=seq(sequence[1],sequence[-1])


        perc2=seq(cut[1],cut[-1])

        overlap_orig=(length(intersect(perc,perc2)))/(sequence[-1]-sequence[1]+1)



        if(((cut)) %overlaps% (sequence) & overlap_orig>=0 ){


            count=count+1

            overlap_outcome[[i]] <- 'yes'

            overlapped_region[[i]]=sequence

            overlapped_percentage[[i]]=overlap_orig


            break


        }


     }


    if(length(overlap_outcome)!=i){

        overlap_outcome[[i]] <- 'no'

        overlapped_region[[i]]=NULL

        overlapped_percentage[[i]]=0



    }



  }

#### for PLAAC comparison, get PLAAC annotations

In [14]:
PLACC_annot <- read.csv(file = './PLAAC/HG38_pep_PLAAC_clean.csv')
head(PLACC_annot)

,SEQid,PRDstart,PRDend,PRDlen
,<chr>,<int>,<int>,<int>
1,ENSP00000371861.4,132,530,399
2,ENSP00000498344.1,92,152,61
3,ENSP00000508087.1,297,695,399
4,ENSP00000384573.1,462,716,255
5,ENSP00000371802.2,132,410,279
6,ENSP00000286835.7,312,402,91


#### check if keys have a PLAAC regions inside their qIDR regions

In [15]:
`%ni%` <- Negate(`%in%`)

In [16]:
count=0
overlap_outcome_PLAAC=list()

for(i in 1:length(int_keys)){
 
     a=int_keys[i]
     idp_values=outcome_IDP_predictor[[a]]
    
    
     if (a %ni% PLACC_annot$SEQid){
         overlap_outcome_PLAAC[[i]] <- 'no'
         next
     }
    
     plaac_values=as.integer(subset(PLACC_annot, SEQid==a,select=c("PRDstart","PRDend")))
     cut=idp_values[1:2]

    
    
    
    for(y in seq(from=1, to=length(plaac_values), by=2)){
        
        

        
    
        #print(cut)
        sequence=c(plaac_values[y]+1,plaac_values[y+1])
        #print(sequence)
        perc=seq(sequence[1],sequence[-1])
        perc2=seq(cut[1],cut[-1])
        overlap_orig=(length(intersect(perc,perc2)))/(sequence[-1]-sequence[1]+1)
        #print(overlap_orig)

        if(((cut)) %overlaps% (sequence) & overlap_orig>=0 ){
            count=count+1
            overlap_outcome_PLAAC[[i]] <- 'yes'

            break
        }
        
        
     }
    
    if(length(overlap_outcome_PLAAC)!=i){
        overlap_outcome_PLAAC[[i]] <- 'no'

    }
  }

   

#### check if keys have a InterProt regions inside their qIDR regions

In [25]:
interpro_indexes<- fromJSON(txt = './Interpro/interpro_indexes_canonical.json')


In [26]:
count=0
overlap_outcome_interprot=list()

for(i in 1:length(int_keys)){
 
     a=int_keys[i]
     #print(a)
     idp_values=outcome_IDP_predictor[[a]]
    
    
       
     if (a %ni% names(interpro_indexes)){
         overlap_outcome_interprot[[i]] <- 'no'
         next
     }
    
     interprot_values=interpro_indexes[[a]]
     cut=idp_values[1:2]


    
    
    for(y in seq(from=1, to=length(interprot_values), by=2)){
    
        #print(cut)
        sequence=c(interprot_values[y]+1,interprot_values[y+1])
        #print(sequence)
        perc=seq(sequence[1],sequence[-1])
        perc2=seq(cut[1],cut[-1])

        if(((cut)) %overlaps% (sequence) ){
            

            count=count+1
            
            
            overlap_outcome_interprot[[i]] <- 'yes'
            
            
            break
        }
        
        
     }
    
    if(length(overlap_outcome_interprot)!=i){
        overlap_outcome_interprot[[i]] <- 'no'
    }
  }

   

### add total disordered residue information

In [27]:
disordered_len_total={}
for(i in 1:length(int_keys)){
    a=int_keys[i]
    len_dis=0
    metapredict_values=json_data_metapredict[[a]]
    if (is.null(metapredict_values)) {
        next}
         
    for(y in seq(from=1, to=length(metapredict_values), by=2)){
        dis=metapredict_values[y+1]-metapredict_values[y]
        len_dis=len_dis+dis
        }
        disordered_len_total[[i]]=len_dis  }
        
        
        
        
    

In [28]:
dataFrame_predict <- transpose(as.data.frame(outcome_IDP_predictor))
rownames(dataFrame_predict)=names(outcome_IDP_predictor)


In [29]:
dataFrame_predict <- subset(dataFrame_predict, rownames(dataFrame_predict) %in% int_keys)
colnames(dataFrame_predict)=c("start(qIDR)","end(qIDR)","length_IDR(qIDR)","min(p-val)","mean_interarrival_window","number_arom_window","number_arom_allseq","len_orig_seq","total_arom_number_qIDR","mean_interarrival_qIDR")
dataFrame_predict$overlap_metapredict=as.character(overlap_outcome)
head(dataFrame_predict)


,start(qIDR),end(qIDR),length_IDR(qIDR),min(p-val),mean_interarrival_window,number_arom_window,number_arom_allseq,len_orig_seq,total_arom_number_qIDR,mean_interarrival_qIDR,overlap_metapredict
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
ENSP00000342812.3,583,737,155,2.797106e-02,8.00000,13,228,2555,16,8.200000,no
ENSP00000354722.2,29,144,116,1.286883e-02,13.71429,8,9,144,8,13.714286,yes
ENSP00000372499.1,373,522,150,2.510095e-02,8.00000,13,40,522,14,8.846154,no
ENSP00000372105.2,227,458,232,5.658611e-03,6.00000,17,46,496,31,7.066667,yes
ENSP00000371861.4,117,558,442,1.226589e-09,3.62963,28,104,558,96,4.315789,no
ENSP00000372154.2,227,458,232,5.658611e-03,6.00000,17,46,496,31,7.066667,yes


In [31]:
dataFrame_predict$overlap_percentage_metapredict=as.character(overlapped_percentage)


In [32]:
dataFrame_predict$disorder_len_total=as.character(disordered_len_total)


### merge dataframes

In [35]:
dataFrame_predict$overlap_interprot=as.character(overlap_outcome_interprot)


In [36]:
dataFrame_predict$overlap_plaac=as.character(overlap_outcome_PLAAC)


In [37]:
head(dataFrame_predict)

,start(qIDR),end(qIDR),length_IDR(qIDR),min(p-val),mean_interarrival_window,number_arom_window,number_arom_allseq,len_orig_seq,total_arom_number_qIDR,mean_interarrival_qIDR,overlap_metapredict,overlap_percentage_metapredict,disorder_len_total,overlap_interprot,overlap_plaac
ENSP00000342812.3,583,737,155,2.797106e-02,8.00000,13,228,2555,16,8.200000,no,0,173,yes,no
ENSP00000354722.2,29,144,116,1.286883e-02,13.71429,8,9,144,8,13.714286,yes,1,51,yes,no
ENSP00000372499.1,373,522,150,2.510095e-02,8.00000,13,40,522,14,8.846154,no,0,29,yes,no
ENSP00000372105.2,227,458,232,5.658611e-03,6.00000,17,46,496,31,7.066667,yes,0.537142857142857,394,no,no
ENSP00000371861.4,117,558,442,1.226589e-09,3.62963,28,104,558,96,4.315789,no,0,26,yes,yes
ENSP00000372154.2,227,458,232,5.658611e-03,6.00000,17,46,496,31,7.066667,yes,0.538461538461538,395,no,no


In [38]:
interprot_values=as.data.frame(as.matrix(interpro_indexes),rownames=FALSE)
interprot_values$ID=rownames(interprot_values)


In [39]:
rownames(interprot_values) <- NULL
colnames(interprot_values)=c("interprot_region","ID")

In [40]:
plaac_values=as.data.frame(as.matrix(PLACC_annot),rownames=FALSE)
plaac_values$plaac_region = paste(plaac_values$PRDstart, plaac_values$PRDend, sep=",")
plaac_values$ID=plaac_values$SEQid


In [41]:
plaac_values=plaac_values[,c(5,6)]

In [42]:
head(plaac_values)

plaac_region,ID
"132, 530",ENSP00000371861.4
"92, 152",ENSP00000498344.1
"297, 695",ENSP00000508087.1
"462, 716",ENSP00000384573.1
"132, 410",ENSP00000371802.2
"312, 402",ENSP00000286835.7


### add gene IDs

In [43]:
data= read.csv("./HG38_pep/HG30_pep_annotation.csv", stringsAsFactors = FALSE)


In [46]:
combined=dataFrame_predict
d <- combined
names <- rownames(d)
rownames(d) <- NULL
data2 <- cbind(names,d)
combined=data2
colnames(combined)[1]="ID"

In [49]:
head(combined)

ID,start(qIDR),end(qIDR),length_IDR(qIDR),min(p-val),mean_interarrival_window,number_arom_window,number_arom_allseq,len_orig_seq,total_arom_number_qIDR,mean_interarrival_qIDR,overlap_metapredict,overlap_percentage_metapredict,disorder_len_total,overlap_interprot,overlap_plaac
ENSP00000342812.3,583,737,155,2.797106e-02,8.00000,13,228,2555,16,8.200000,no,0,173,yes,no
ENSP00000354722.2,29,144,116,1.286883e-02,13.71429,8,9,144,8,13.714286,yes,1,51,yes,no
ENSP00000372499.1,373,522,150,2.510095e-02,8.00000,13,40,522,14,8.846154,no,0,29,yes,no
ENSP00000372105.2,227,458,232,5.658611e-03,6.00000,17,46,496,31,7.066667,yes,0.537142857142857,394,no,no
ENSP00000371861.4,117,558,442,1.226589e-09,3.62963,28,104,558,96,4.315789,no,0,26,yes,yes
ENSP00000372154.2,227,458,232,5.658611e-03,6.00000,17,46,496,31,7.066667,yes,0.538461538461538,395,no,no


In [50]:
namesofProt=data[data$ID %in% combined$ID,c("gene_symbol","ID")]

In [51]:
combined_annot=(merge(combined, namesofProt, by = "ID"))                     # Merge data according to row namesroq

In [52]:
head(combined_annot)

ID,start(qIDR),end(qIDR),length_IDR(qIDR),min(p-val),mean_interarrival_window,number_arom_window,number_arom_allseq,len_orig_seq,total_arom_number_qIDR,mean_interarrival_qIDR,overlap_metapredict,overlap_percentage_metapredict,disorder_len_total,overlap_interprot,overlap_plaac,gene_symbol
ENSP00000000412.3,109,209,101,0.31750221,9.600000,11,27,277,11,9.600000,no,0,22,yes,no,M6PR
ENSP00000000442.6,13,135,123,0.12688509,4.666667,4,21,423,5,6.000000,yes,0.818181818181818,66,yes,no,ESRRA
ENSP00000001008.4,153,282,130,0.04322908,8.166667,13,39,459,15,8.571429,no,0,63,yes,no,FKBP4
ENSP00000002829.3,39,175,137,0.01490606,10.444444,10,74,785,13,9.083333,no,0,34,yes,no,SEMA3F
ENSP00000003302.4,147,285,139,0.02580273,8.909091,12,88,1077,15,8.357143,no,0,101,yes,no,USP28
ENSP00000005226.7,667,825,159,0.03720216,5.375000,9,56,899,12,12.727273,yes,0.115384615384615,182,yes,no,USH1C


In [53]:
#order based on p-value
df <-combined_annot[order(combined_annot$'min(p-val)'),]

In [55]:
head(df)

,ID,start(qIDR),end(qIDR),length_IDR(qIDR),min(p-val),mean_interarrival_window,number_arom_window,number_arom_allseq,len_orig_seq,total_arom_number_qIDR,mean_interarrival_qIDR,overlap_metapredict,overlap_percentage_metapredict,disorder_len_total,overlap_interprot,overlap_plaac,gene_symbol
6124,ENSP00000371861.4,117,558,442,1.226589e-09,3.629630,28,104,558,96,4.315789,no,0,26,yes,yes,DAZ2
6763,ENSP00000384573.1,442,744,303,1.226589e-09,3.629630,28,91,744,61,4.433333,no,0,84,yes,yes,DAZ1
9380,ENSP00000508087.1,279,723,445,1.226589e-09,3.629630,28,116,723,97,4.270833,no,0,55,yes,yes,DAZ4
6119,ENSP00000371802.2,117,438,322,7.424618e-09,3.769231,27,74,438,66,4.461538,no,0,26,yes,yes,DAZ3
6431,ENSP00000378507.4,1,213,213,1.780978e-08,5.500000,19,57,371,37,5.583333,yes,1,139,yes,yes,GRINA
8966,ENSP00000495481.1,146,471,326,3.303475e-08,3.161290,32,316,3530,64,4.746032,no,0,684,no,no,MYO15A


In [ ]:
## add repeat strength of proteins to the table

In [57]:


repeat_output<- fromJSON("./repeats/repeat_output_humanproteom.json")


In [58]:
dataFrame_repeat <- transpose(as.data.frame(repeat_output))


In [59]:
dataFrame_repeat$ID=names(repeat_output)


In [60]:
colnames(dataFrame_repeat)=c("repeat_strength","ID")

In [61]:
metapredict_values=as.data.frame(as.matrix(json_data_metapredict),rownames=FALSE)
metapredict_values$ID=rownames(metapredict_values)
rownames(metapredict_values) <- NULL

In [62]:
colnames(metapredict_values)=c("metapredict_IDR_region","ID")

In [64]:
merged_df=merge(df, dataFrame_repeat, by="ID") # NA's match


In [66]:
merged_df_2=merge(merged_df, metapredict_values, by="ID") # NA's match


In [68]:
df_list <- list(merged_df_2, interprot_values, plaac_values)
df_all_merged=Reduce(function(x, y) merge(x, y, by="ID", all.x = TRUE), df_list, accumulate=FALSE)

In [70]:
merged_df_2 <- df_all_merged[order(df_all_merged$`min(p-val)`),] 

merged_df_2$rank=seq(1,length(merged_df_2$ID))

In [73]:
merged_df_2$fract_arom=as.numeric(merged_df_2$number_arom_allseq)/as.numeric(merged_df_2$len_orig_seq)

In [75]:
merged_df_2$metapredict_IDR_region <- as.character(merged_df_2$metapredict_IDR_region)

### add annotation for PLDs, TFs and RNA binding proteins 

In [78]:
TFs_annot= read.csv("./TFs/AnnotationTFs.csv", stringsAsFactors = FALSE)


In [80]:
RBPs_annot= read.csv("./RNAbindingProteins/RBP_HS_keys.csv", stringsAsFactors = FALSE)


UNIQUE,ID,Name,Description,MEF.IC,mESC.nIC,RAW264.7.IC,mESC.IC,HL.1.IC,HL.1.RBDmap,Enzyme,Metabolism,X.Metabolic.Enzyme.
,,,,Mm_Boucas2015,Mm_He2016,Mm_Liepelt2016,Mm_Kwon2013,Mm_Liao2016_IC,Mm_Liao2016_RBDmap,NA,NA,NA
0610030E20Rik,ENSMUSG00000058706;ENSMUSG00000108680,0610030E20Rik,RIKEN cDNA 0610030E20 gene,no,no,YES,no,no,no,FALSE,FALSE,FALSE
1110004F10Rik,ENSMUSG00000030663,1110004F10Rik,RIKEN cDNA 1110004F10 gene,no,no,YES,no,no,YES,FALSE,FALSE,FALSE
1110008L16Rik,ENSMUSG00000021023,1110008L16Rik,RIKEN cDNA 1110008L16 gene,no,no,no,no,YES,no,FALSE,FALSE,FALSE
1110037F02Rik,ENSMUSG00000040720,1110037F02Rik,RIKEN cDNA 1110037F02 gene,no,no,no,no,YES,no,FALSE,FALSE,FALSE
1700009N14Rik,ENSMUSG00000028287,1700009N14Rik,RIKEN cDNA 1700009N14 gene,no,no,no,no,YES,no,FALSE,FALSE,FALSE


In [81]:
RPBs_keys=tail(as.vector(RBPs_annot$UNIQUE), -1)

In [82]:
rbp_annot=list()
tf_annot=list()
plaac_annot=list()

for (i in 1:length(merged_df_2$gene_symbol)) {
    if(merged_df_2$gene_symbol[[i]]%in% toupper(RPBs_keys)){
        rbp_annot[i]="yes"
    }
    else if(merged_df_2$gene_symbol[[i]] %ni% toupper(RPBs_keys)){
        rbp_annot[i]="no"
    } 
        
    if(merged_df_2$gene_symbol[[i]]%in% toupper(TFs_annot$Name)){
        tf_annot[i]="yes"
    }
    else if(merged_df_2$gene_symbol[[i]] %ni% toupper(TFs_annot$Name)){
        tf_annot[i]="no"
    } 
        
        if(merged_df_2$ID[[i]] %in% (plaac_values$ID)){
        plaac_annot[i]="yes"
    }
    else if(merged_df_2$ID[[i]] %ni% (plaac_values$ID)){
        plaac_annot[i]="no"
    }     
  #print(merged_df_2$gene_symbol[[i]])
}

In [84]:
merged_df_2$plaac_annot=as.character(plaac_annot)
merged_df_2$rbp_annot=as.character(rbp_annot)
merged_df_2$tf_annot=as.character(tf_annot)


In [269]:
merged_df_2$interprot_region=as.character(merged_df_2$interprot_region)

In [270]:
write_xlsx(merged_df_2,"MasterTable_canonicalHuman_qIDR.xlsx")